# Feature Engineering

## Read file

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import math
import scipy.spatial
import warnings
import sklearn as sk
import operator
import numpy as np

warnings.simplefilter("ignore")

# Enable inline mode for matplotlib so that Jupyter displays graphs
%matplotlib inline

pd.__version__ #print which version of pandas you're using

'0.24.2'

In [2]:
raw_data = pd.read_csv('h1b.csv')
print(raw_data.shape)
raw_data.head()

(1279010, 50)


,CASE_NUMBER,CASE_STATUS,CASE_SUBMITTED,VISA_CLASS,EMPLOYMENT_START_DATE,EMPLOYMENT_END_DATE,EMPLOYER_NAME,EMPLOYER_BUSINESS_DBA,EMPLOYER_ADDRESS,EMPLOYER_CITY,...,WAGE_UNIT_OF_PAY,H1B_DEPENDENT,WILLFUL_VIOLATOR,SUPPORT_H1B,LABOR_CON_AGREE,PUBLIC_DISCLOSURE_LOCATION,WORKSITE_CITY,WORKSITE_COUNTY,WORKSITE_STATE,WORKSITE_POSTAL_CODE
0,I-200-16055-173457,CERTIFIED-WITHDRAWN,2,H-1B,16,19,DISCOVER PRODUCTS INC.,NaN,2500 LAKE COOK ROAD,RIVERWOODS,...,Year,N,N,NaN,Y,NaN,RIVERWOODS,LAKE,IL,60015
1,I-200-16064-557834,CERTIFIED-WITHDRAWN,3,H-1B,16,19,DFS SERVICES LLC,NaN,2500 LAKE COOK ROAD,RIVERWOODS,...,Year,N,N,NaN,Y,NaN,RIVERWOODS,LAKE,IL,60015
2,I-200-16063-996093,CERTIFIED-WITHDRAWN,3,H-1B,16,19,EASTBANC TECHNOLOGIES LLC,NaN,1211 31ST ST. NW,WASHINGTON,...,Year,Y,N,Y,NaN,NaN,WASHINGTON,NaN,DC,20007
3,I-200-16272-196340,WITHDRAWN,9,H-1B,17,20,INFO SERVICES LLC,NaN,17177 NORTH LAUREL PARK DR,LIVONIA,...,Year,Y,N,Y,NaN,NaN,JERSEY CITY,HUDSON,NJ,07302
4,I-200-15053-636744,CERTIFIED-WITHDRAWN,2,H-1B,15,18,BB&T CORPORATION,NaN,223 WEST NASH STREET,WILSON,...,Year,N,N,NaN,Y,NaN,NEW YORK,NEW YORK,NY,10036


## Only keep Visa_class = H-1B

In [3]:
h1b_data = raw_data[raw_data['VISA_CLASS'] == 'H-1B']
print(h1b_data.shape)

(1249823, 50)


In [4]:
h1b_data.columns

Index(['CASE_NUMBER', 'CASE_STATUS', 'CASE_SUBMITTED', 'VISA_CLASS',
       'EMPLOYMENT_START_DATE', 'EMPLOYMENT_END_DATE', 'EMPLOYER_NAME',
       'EMPLOYER_BUSINESS_DBA', 'EMPLOYER_ADDRESS', 'EMPLOYER_CITY',
       'EMPLOYER_STATE', 'EMPLOYER_POSTAL_CODE', 'EMPLOYER_COUNTRY',
       'EMPLOYER_PROVINCE', 'EMPLOYER_PHONE', 'EMPLOYER_PHONE_EXT',
       'AGENT_REPRESENTING_EMPLOYER', 'AGENT_ATTORNEY_NAME',
       'AGENT_ATTORNEY_CITY', 'AGENT_ATTORNEY_STATE', 'JOB_TITLE', 'SOC_CODE',
       'SOC_NAME', 'NAICS_CODE', 'TOTAL_WORKERS', 'NEW_EMPLOYMENT',
       'CONTINUED_EMPLOYMENT', 'CHANGE_PREVIOUS_EMPLOYMENT',
       'NEW_CONCURRENT_EMPLOYMENT', 'CHANGE_EMPLOYER', 'AMENDED_PETITION',
       'FULL_TIME_POSITION', 'PREVAILING_WAGE', 'PW_UNIT_OF_PAY',
       'PW_WAGE_LEVEL', 'PW_SOURCE', 'PW_SOURCE_YEAR', 'PW_SOURCE_OTHER',
       'WAGE_RATE_OF_PAY_FROM', 'WAGE_RATE_OF_PAY_TO', 'WAGE_UNIT_OF_PAY',
       'H1B_DEPENDENT', 'WILLFUL_VIOLATOR', 'SUPPORT_H1B', 'LABOR_CON_AGREE',
       'PUBLIC_D

## Create label and drop some features

In [5]:
status = h1b_data['CASE_STATUS']
label = []
for s in status:
    if s == 'CERTIFIED':
        label.append(1)
    else:
        label.append(0)
h1b_data['label'] = label
h1b_data = h1b_data.drop('CASE_STATUS', axis=1)
h1b_data = h1b_data.drop('CASE_NUMBER', axis=1)
h1b_data = h1b_data.drop('CASE_SUBMITTED', axis=1)
h1b_data = h1b_data.drop('VISA_CLASS', axis=1)
h1b_data.head()

,EMPLOYMENT_START_DATE,EMPLOYMENT_END_DATE,EMPLOYER_NAME,EMPLOYER_BUSINESS_DBA,EMPLOYER_ADDRESS,EMPLOYER_CITY,EMPLOYER_STATE,EMPLOYER_POSTAL_CODE,EMPLOYER_COUNTRY,EMPLOYER_PROVINCE,...,H1B_DEPENDENT,WILLFUL_VIOLATOR,SUPPORT_H1B,LABOR_CON_AGREE,PUBLIC_DISCLOSURE_LOCATION,WORKSITE_CITY,WORKSITE_COUNTY,WORKSITE_STATE,WORKSITE_POSTAL_CODE,label
0,16,19,DISCOVER PRODUCTS INC.,NaN,2500 LAKE COOK ROAD,RIVERWOODS,IL,60015,UNITED STATES OF AMERICA,NaN,...,N,N,NaN,Y,NaN,RIVERWOODS,LAKE,IL,60015,0
1,16,19,DFS SERVICES LLC,NaN,2500 LAKE COOK ROAD,RIVERWOODS,IL,60015,UNITED STATES OF AMERICA,NaN,...,N,N,NaN,Y,NaN,RIVERWOODS,LAKE,IL,60015,0
2,16,19,EASTBANC TECHNOLOGIES LLC,NaN,1211 31ST ST. NW,WASHINGTON,DC,20007,UNITED STATES OF AMERICA,NaN,...,Y,N,Y,NaN,NaN,WASHINGTON,NaN,DC,20007,0
3,17,20,INFO SERVICES LLC,NaN,17177 NORTH LAUREL PARK DR,LIVONIA,MI,48152,UNITED STATES OF AMERICA,NaN,...,Y,N,Y,NaN,NaN,JERSEY CITY,HUDSON,NJ,07302,0
4,15,18,BB&T CORPORATION,NaN,223 WEST NASH STREET,WILSON,NC,27893,UNITED STATES OF AMERICA,NaN,...,N,N,NaN,Y,NaN,NEW YORK,NEW YORK,NY,10036,0


## Dealing with NaN

### fill 'N' on yes/no columns

In [6]:
h1b_data.columns

Index(['EMPLOYMENT_START_DATE', 'EMPLOYMENT_END_DATE', 'EMPLOYER_NAME',
       'EMPLOYER_BUSINESS_DBA', 'EMPLOYER_ADDRESS', 'EMPLOYER_CITY',
       'EMPLOYER_STATE', 'EMPLOYER_POSTAL_CODE', 'EMPLOYER_COUNTRY',
       'EMPLOYER_PROVINCE', 'EMPLOYER_PHONE', 'EMPLOYER_PHONE_EXT',
       'AGENT_REPRESENTING_EMPLOYER', 'AGENT_ATTORNEY_NAME',
       'AGENT_ATTORNEY_CITY', 'AGENT_ATTORNEY_STATE', 'JOB_TITLE', 'SOC_CODE',
       'SOC_NAME', 'NAICS_CODE', 'TOTAL_WORKERS', 'NEW_EMPLOYMENT',
       'CONTINUED_EMPLOYMENT', 'CHANGE_PREVIOUS_EMPLOYMENT',
       'NEW_CONCURRENT_EMPLOYMENT', 'CHANGE_EMPLOYER', 'AMENDED_PETITION',
       'FULL_TIME_POSITION', 'PREVAILING_WAGE', 'PW_UNIT_OF_PAY',
       'PW_WAGE_LEVEL', 'PW_SOURCE', 'PW_SOURCE_YEAR', 'PW_SOURCE_OTHER',
       'WAGE_RATE_OF_PAY_FROM', 'WAGE_RATE_OF_PAY_TO', 'WAGE_UNIT_OF_PAY',
       'H1B_DEPENDENT', 'WILLFUL_VIOLATOR', 'SUPPORT_H1B', 'LABOR_CON_AGREE',
       'PUBLIC_DISCLOSURE_LOCATION', 'WORKSITE_CITY', 'WORKSITE_COUNTY',
       'WORK

In [7]:
YN_columns = ['FULL_TIME_POSITION', 'AGENT_REPRESENTING_EMPLOYER', 'H1B_DEPENDENT',
             'WILLFUL_VIOLATOR', 'SUPPORT_H1B', 'LABOR_CON_AGREE']

In [8]:
for c in YN_columns:
    h1b_data[c] = h1b_data[c].fillna('N')
h1b_data.head(10)

,EMPLOYMENT_START_DATE,EMPLOYMENT_END_DATE,EMPLOYER_NAME,EMPLOYER_BUSINESS_DBA,EMPLOYER_ADDRESS,EMPLOYER_CITY,EMPLOYER_STATE,EMPLOYER_POSTAL_CODE,EMPLOYER_COUNTRY,EMPLOYER_PROVINCE,...,H1B_DEPENDENT,WILLFUL_VIOLATOR,SUPPORT_H1B,LABOR_CON_AGREE,PUBLIC_DISCLOSURE_LOCATION,WORKSITE_CITY,WORKSITE_COUNTY,WORKSITE_STATE,WORKSITE_POSTAL_CODE,label
0,16,19,DISCOVER PRODUCTS INC.,NaN,2500 LAKE COOK ROAD,RIVERWOODS,IL,60015,UNITED STATES OF AMERICA,NaN,...,N,N,N,Y,NaN,RIVERWOODS,LAKE,IL,60015,0
1,16,19,DFS SERVICES LLC,NaN,2500 LAKE COOK ROAD,RIVERWOODS,IL,60015,UNITED STATES OF AMERICA,NaN,...,N,N,N,Y,NaN,RIVERWOODS,LAKE,IL,60015,0
2,16,19,EASTBANC TECHNOLOGIES LLC,NaN,1211 31ST ST. NW,WASHINGTON,DC,20007,UNITED STATES OF AMERICA,NaN,...,Y,N,Y,N,NaN,WASHINGTON,NaN,DC,20007,0
3,17,20,INFO SERVICES LLC,NaN,17177 NORTH LAUREL PARK DR,LIVONIA,MI,48152,UNITED STATES OF AMERICA,NaN,...,Y,N,Y,N,NaN,JERSEY CITY,HUDSON,NJ,07302,0
4,15,18,BB&T CORPORATION,NaN,223 WEST NASH STREET,WILSON,NC,27893,UNITED STATES OF AMERICA,NaN,...,N,N,N,Y,NaN,NEW YORK,NEW YORK,NY,10036,0
5,15,18,"SUNTRUST BANKS, INC.",NaN,"303 PEACHTREE STREET, NE",ATLANTA,GA,30308,UNITED STATES OF AMERICA,NaN,...,N,N,N,Y,NaN,ATLANTA,FULTON,GA,30303,0
6,16,19,CITADEL INFORMATION SERVICES INC.,CITADEL,33 WOOD AVENUE SOUTH,ISELIN,NJ,08830,UNITED STATES OF AMERICA,NaN,...,Y,N,Y,Y,NaN,EDISON,MIDDLESEX,NJ,08837,0
7,16,19,CITADEL INFORMATION SERVICES INC.,CITADEL,33 WOOD AVENUE SOUTH,ISELIN,NJ,08830,UNITED STATES OF AMERICA,NaN,...,Y,N,Y,Y,NaN,EDISON,MIDDLESEX,NJ,08837,0
8,16,19,CITADEL INFORMATION SERVICES INC.,CITADEL,33 WOOD AVENUE SOUTH,ISELIN,NJ,08830,UNITED STATES OF AMERICA,NaN,...,Y,N,Y,Y,NaN,NEW YORK,NEW YORK,NY,10005,0
9,16,19,CITADEL INFORMATION SERVICES INC.,CITADEL,33 WOOD AVENUE SOUTH,ISELIN,NJ,08830,UNITED STATES OF AMERICA,NaN,...,Y,N,Y,Y,NaN,ISELIN,MIDDLESEX,NJ,08830,0


### remove columns that have so many NaN

In [9]:
def nan_percent(col):
    count = 0.0
    na = pd.isna(col)
    for i in na:
        if i:
            count += 1.0
    return count/len(col)

In [10]:
reduce_features = []
for c in h1b_data.columns:
    if nan_percent(h1b_data[c]) > 0.3:
        reduce_features.append(c)
print(reduce_features)

['EMPLOYER_BUSINESS_DBA', 'EMPLOYER_PROVINCE', 'EMPLOYER_PHONE_EXT', 'AGENT_ATTORNEY_CITY', 'AGENT_ATTORNEY_STATE', 'PUBLIC_DISCLOSURE_LOCATION']


In [11]:
reduced_data = h1b_data
for c in reduce_features:
    reduced_data = reduced_data.drop(c, axis=1)

In [12]:
print(reduced_data.columns)
reduced_data.head()

Index(['EMPLOYMENT_START_DATE', 'EMPLOYMENT_END_DATE', 'EMPLOYER_NAME',
       'EMPLOYER_ADDRESS', 'EMPLOYER_CITY', 'EMPLOYER_STATE',
       'EMPLOYER_POSTAL_CODE', 'EMPLOYER_COUNTRY', 'EMPLOYER_PHONE',
       'AGENT_REPRESENTING_EMPLOYER', 'AGENT_ATTORNEY_NAME', 'JOB_TITLE',
       'SOC_CODE', 'SOC_NAME', 'NAICS_CODE', 'TOTAL_WORKERS', 'NEW_EMPLOYMENT',
       'CONTINUED_EMPLOYMENT', 'CHANGE_PREVIOUS_EMPLOYMENT',
       'NEW_CONCURRENT_EMPLOYMENT', 'CHANGE_EMPLOYER', 'AMENDED_PETITION',
       'FULL_TIME_POSITION', 'PREVAILING_WAGE', 'PW_UNIT_OF_PAY',
       'PW_WAGE_LEVEL', 'PW_SOURCE', 'PW_SOURCE_YEAR', 'PW_SOURCE_OTHER',
       'WAGE_RATE_OF_PAY_FROM', 'WAGE_RATE_OF_PAY_TO', 'WAGE_UNIT_OF_PAY',
       'H1B_DEPENDENT', 'WILLFUL_VIOLATOR', 'SUPPORT_H1B', 'LABOR_CON_AGREE',
       'WORKSITE_CITY', 'WORKSITE_COUNTY', 'WORKSITE_STATE',
       'WORKSITE_POSTAL_CODE', 'label'],
      dtype='object')


,EMPLOYMENT_START_DATE,EMPLOYMENT_END_DATE,EMPLOYER_NAME,EMPLOYER_ADDRESS,EMPLOYER_CITY,EMPLOYER_STATE,EMPLOYER_POSTAL_CODE,EMPLOYER_COUNTRY,EMPLOYER_PHONE,AGENT_REPRESENTING_EMPLOYER,...,WAGE_UNIT_OF_PAY,H1B_DEPENDENT,WILLFUL_VIOLATOR,SUPPORT_H1B,LABOR_CON_AGREE,WORKSITE_CITY,WORKSITE_COUNTY,WORKSITE_STATE,WORKSITE_POSTAL_CODE,label
0,16,19,DISCOVER PRODUCTS INC.,2500 LAKE COOK ROAD,RIVERWOODS,IL,60015,UNITED STATES OF AMERICA,2244050900,Y,...,Year,N,N,N,Y,RIVERWOODS,LAKE,IL,60015,0
1,16,19,DFS SERVICES LLC,2500 LAKE COOK ROAD,RIVERWOODS,IL,60015,UNITED STATES OF AMERICA,2244050900,Y,...,Year,N,N,N,Y,RIVERWOODS,LAKE,IL,60015,0
2,16,19,EASTBANC TECHNOLOGIES LLC,1211 31ST ST. NW,WASHINGTON,DC,20007,UNITED STATES OF AMERICA,2022953000,Y,...,Year,Y,N,Y,N,WASHINGTON,NaN,DC,20007,0
3,17,20,INFO SERVICES LLC,17177 NORTH LAUREL PARK DR,LIVONIA,MI,48152,UNITED STATES OF AMERICA,7343776007,N,...,Year,Y,N,Y,N,JERSEY CITY,HUDSON,NJ,07302,0
4,15,18,BB&T CORPORATION,223 WEST NASH STREET,WILSON,NC,27893,UNITED STATES OF AMERICA,2522462031,Y,...,Year,N,N,N,Y,NEW YORK,NEW YORK,NY,10036,0


## Find correlations between features

In [13]:
corr = reduced_data.corr(method='pearson')
corr.style.background_gradient(cmap='coolwarm', axis=None)

,EMPLOYMENT_START_DATE,EMPLOYMENT_END_DATE,TOTAL_WORKERS,NEW_EMPLOYMENT,CONTINUED_EMPLOYMENT,CHANGE_PREVIOUS_EMPLOYMENT,NEW_CONCURRENT_EMPLOYMENT,CHANGE_EMPLOYER,AMENDED_PETITION,PREVAILING_WAGE,PW_SOURCE_YEAR,WAGE_RATE_OF_PAY_FROM,WAGE_RATE_OF_PAY_TO,label
EMPLOYMENT_START_DATE,1,0.849882,0.014669,0.0251993,0.0215066,0.0037496,0.0101066,-0.0125374,-0.0276801,0.00352235,0.146545,0.00321852,0.00449765,0.293222
EMPLOYMENT_END_DATE,0.849882,1,0.0351546,0.0407578,0.0260076,0.013216,0.0111982,0.0104315,-0.0268224,0.00466301,0.126674,0.00455481,0.0166969,0.269941
TOTAL_WORKERS,0.014669,0.0351546,1,0.762186,0.442102,0.341229,0.208877,0.425634,0.438796,0.000724649,0.00167033,-0.000422933,0.116348,0.0405993
NEW_EMPLOYMENT,0.0251993,0.0407578,0.762186,1,0.0389159,0.0242239,0.0445264,0.0360508,0.0320113,-0.000345467,-0.00342098,-0.00144235,0.0695137,0.0125169
CONTINUED_EMPLOYMENT,0.0215066,0.0260076,0.442102,0.0389159,1,0.112569,0.12438,0.183142,0.161626,0.00115647,0.00366413,0.000868415,0.0510347,0.0298305
CHANGE_PREVIOUS_EMPLOYMENT,0.0037496,0.013216,0.341229,0.0242239,0.112569,1,0.185354,0.181895,0.12978,0.000630291,0.00307247,0.000244176,0.0876169,0.0235856
NEW_CONCURRENT_EMPLOYMENT,0.0101066,0.0111982,0.208877,0.0445264,0.12438,0.185354,1,0.13627,0.107211,-0.00266486,0.00618635,-0.00317233,0.0131228,0.0122649
CHANGE_EMPLOYER,-0.0125374,0.0104315,0.425634,0.0360508,0.183142,0.181895,0.13627,1,0.186508,0.00203384,0.00569125,0.00200916,0.0632924,0.0265861
AMENDED_PETITION,-0.0276801,-0.0268224,0.438796,0.0320113,0.161626,0.12978,0.107211,0.186508,1,0.000376054,0.00257725,-0.000164678,0.0553423,0.0389782
PREVAILING_WAGE,0.00352235,0.00466301,0.000724649,-0.000345467,0.00115647,0.000630291,-0.00266486,0.00203384,0.000376054,1,0.000800088,0.979902,0.00921472,-0.00106845


In [14]:
corr = reduced_data.corr(method='spearman')
corr.style.background_gradient(cmap='coolwarm', axis=None)

,EMPLOYMENT_START_DATE,EMPLOYMENT_END_DATE,TOTAL_WORKERS,NEW_EMPLOYMENT,CONTINUED_EMPLOYMENT,CHANGE_PREVIOUS_EMPLOYMENT,NEW_CONCURRENT_EMPLOYMENT,CHANGE_EMPLOYER,AMENDED_PETITION,PREVAILING_WAGE,PW_SOURCE_YEAR,WAGE_RATE_OF_PAY_FROM,WAGE_RATE_OF_PAY_TO,label
EMPLOYMENT_START_DATE,1,0.903474,0.0118207,0.050929,0.0522945,0.00995136,-0.000910407,-0.0406764,-0.0851006,0.163626,0.764677,0.129508,0.00203678,0.206846
EMPLOYMENT_END_DATE,0.903474,1,0.0397506,0.0893468,0.0413037,0.0167221,-0.000975015,-0.0105968,-0.123818,0.186002,0.693085,0.153124,0.00438074,0.210502
TOTAL_WORKERS,0.0118207,0.0397506,1,0.235316,0.175429,0.231228,0.182416,0.24123,0.245715,-0.00192016,0.00906572,-0.0363239,0.104352,0.0552193
NEW_EMPLOYMENT,0.050929,0.0893468,0.235316,1,-0.317629,-0.113655,0.0589862,-0.254345,-0.260531,-0.229992,-0.0818801,-0.259839,-0.0251327,-0.038989
CONTINUED_EMPLOYMENT,0.0522945,0.0413037,0.175429,-0.317629,1,0.0349869,0.108158,-0.0273875,-0.0497652,0.0864748,0.0404167,0.0857536,0.0308454,0.035102
CHANGE_PREVIOUS_EMPLOYMENT,0.00995136,0.0167221,0.231228,-0.113655,0.0349869,1,0.198676,0.0819781,0.0232945,0.0386708,0.0284349,0.0314247,0.0649396,0.0370345
NEW_CONCURRENT_EMPLOYMENT,-0.000910407,-0.000975015,0.182416,0.0589862,0.108158,0.198676,1,0.113001,0.113009,-0.0596092,0.0328563,-0.0660428,0.0678883,0.0145339
CHANGE_EMPLOYER,-0.0406764,-0.0105968,0.24123,-0.254345,-0.0273875,0.0819781,0.113001,1,-0.00977996,0.177399,0.057536,0.186391,0.0636551,0.0245661
AMENDED_PETITION,-0.0851006,-0.123818,0.245715,-0.260531,-0.0497652,0.0232945,0.113009,-0.00977996,1,0.0404718,0.0232385,0.0238315,0.0203435,0.0550398
PREVAILING_WAGE,0.163626,0.186002,-0.00192016,-0.229992,0.0864748,0.0386708,-0.0596092,0.177399,0.0404718,1,0.165462,0.9135,0.154889,0.0658438


In [15]:
corr = reduced_data.corr(method='kendall')
corr.style.background_gradient(cmap='coolwarm', axis=None)

,EMPLOYMENT_START_DATE,EMPLOYMENT_END_DATE,TOTAL_WORKERS,NEW_EMPLOYMENT,CONTINUED_EMPLOYMENT,CHANGE_PREVIOUS_EMPLOYMENT,NEW_CONCURRENT_EMPLOYMENT,CHANGE_EMPLOYER,AMENDED_PETITION,PREVAILING_WAGE,PW_SOURCE_YEAR,WAGE_RATE_OF_PAY_FROM,WAGE_RATE_OF_PAY_TO,label
EMPLOYMENT_START_DATE,1,0.884915,0.0111651,0.0480055,0.0499368,0.00955682,-0.000878123,-0.0388452,-0.0811751,0.129583,0.724848,0.102805,0.00183007,0.199827
EMPLOYMENT_END_DATE,0.884915,1,0.0370439,0.0830506,0.0388403,0.0158224,-0.000926883,-0.009952,-0.116323,0.145063,0.645985,0.119522,0.00387594,0.200449
TOTAL_WORKERS,0.0111651,0.0370439,1,0.228633,0.171467,0.226927,0.178977,0.236229,0.239958,-0.00149877,0.00837931,-0.0290678,0.0970224,0.0541465
NEW_EMPLOYMENT,0.0480055,0.0830506,0.228633,1,-0.312384,-0.111603,0.0577185,-0.249796,-0.255219,-0.183731,-0.0759995,-0.207567,-0.0227947,-0.038147
CONTINUED_EMPLOYMENT,0.0499368,0.0388403,0.171467,-0.312384,1,0.0343159,0.107011,-0.0280304,-0.0498635,0.0699258,0.0379024,0.0693973,0.0289107,0.0347164
CHANGE_PREVIOUS_EMPLOYMENT,0.00955682,0.0158224,0.226927,-0.111603,0.0343159,1,0.197622,0.0809979,0.0228084,0.0314767,0.0268193,0.0256163,0.0611892,0.0368191
NEW_CONCURRENT_EMPLOYMENT,-0.000878123,-0.000926883,0.178977,0.0577185,0.107011,0.197622,1,0.111797,0.111647,-0.0486576,0.0311468,-0.0540178,0.0641921,0.0145214
CHANGE_EMPLOYER,-0.0388452,-0.009952,0.236229,-0.249796,-0.0280304,0.0809979,0.111797,1,-0.0103461,0.143244,0.05401,0.150597,0.0596912,0.0243102
AMENDED_PETITION,-0.0811751,-0.116323,0.239958,-0.255219,-0.0498635,0.0228084,0.111647,-0.0103461,1,0.0328345,0.0217827,0.0193341,0.0190656,0.0543894
PREVAILING_WAGE,0.129583,0.145063,-0.00149877,-0.183731,0.0699258,0.0314767,-0.0486576,0.143244,0.0328345,1,0.129798,0.79749,0.127527,0.0537796


### based on the correlation matrices, Prevailing wage and wage_rate_of_pay_from have a high correlation, so we can remove one

In [13]:
reduced_data = reduced_data.drop('WAGE_RATE_OF_PAY_FROM', axis=1)
reduced_data.head()

,EMPLOYMENT_START_DATE,EMPLOYMENT_END_DATE,EMPLOYER_NAME,EMPLOYER_ADDRESS,EMPLOYER_CITY,EMPLOYER_STATE,EMPLOYER_POSTAL_CODE,EMPLOYER_COUNTRY,EMPLOYER_PHONE,AGENT_REPRESENTING_EMPLOYER,...,WAGE_UNIT_OF_PAY,H1B_DEPENDENT,WILLFUL_VIOLATOR,SUPPORT_H1B,LABOR_CON_AGREE,WORKSITE_CITY,WORKSITE_COUNTY,WORKSITE_STATE,WORKSITE_POSTAL_CODE,label
0,16,19,DISCOVER PRODUCTS INC.,2500 LAKE COOK ROAD,RIVERWOODS,IL,60015,UNITED STATES OF AMERICA,2244050900,Y,...,Year,N,N,N,Y,RIVERWOODS,LAKE,IL,60015,0
1,16,19,DFS SERVICES LLC,2500 LAKE COOK ROAD,RIVERWOODS,IL,60015,UNITED STATES OF AMERICA,2244050900,Y,...,Year,N,N,N,Y,RIVERWOODS,LAKE,IL,60015,0
2,16,19,EASTBANC TECHNOLOGIES LLC,1211 31ST ST. NW,WASHINGTON,DC,20007,UNITED STATES OF AMERICA,2022953000,Y,...,Year,Y,N,Y,N,WASHINGTON,NaN,DC,20007,0
3,17,20,INFO SERVICES LLC,17177 NORTH LAUREL PARK DR,LIVONIA,MI,48152,UNITED STATES OF AMERICA,7343776007,N,...,Year,Y,N,Y,N,JERSEY CITY,HUDSON,NJ,07302,0
4,15,18,BB&T CORPORATION,223 WEST NASH STREET,WILSON,NC,27893,UNITED STATES OF AMERICA,2522462031,Y,...,Year,N,N,N,Y,NEW YORK,NEW YORK,NY,10036,0


### employment start date and employment end date also have high correlation

In [14]:
reduced_data = reduced_data.drop('EMPLOYMENT_START_DATE', axis=1)
reduced_data.head()

,EMPLOYMENT_END_DATE,EMPLOYER_NAME,EMPLOYER_ADDRESS,EMPLOYER_CITY,EMPLOYER_STATE,EMPLOYER_POSTAL_CODE,EMPLOYER_COUNTRY,EMPLOYER_PHONE,AGENT_REPRESENTING_EMPLOYER,AGENT_ATTORNEY_NAME,...,WAGE_UNIT_OF_PAY,H1B_DEPENDENT,WILLFUL_VIOLATOR,SUPPORT_H1B,LABOR_CON_AGREE,WORKSITE_CITY,WORKSITE_COUNTY,WORKSITE_STATE,WORKSITE_POSTAL_CODE,label
0,19,DISCOVER PRODUCTS INC.,2500 LAKE COOK ROAD,RIVERWOODS,IL,60015,UNITED STATES OF AMERICA,2244050900,Y,"ELLSWORTH, CHAD",...,Year,N,N,N,Y,RIVERWOODS,LAKE,IL,60015,0
1,19,DFS SERVICES LLC,2500 LAKE COOK ROAD,RIVERWOODS,IL,60015,UNITED STATES OF AMERICA,2244050900,Y,"ELLSWORTH, CHAD",...,Year,N,N,N,Y,RIVERWOODS,LAKE,IL,60015,0
2,19,EASTBANC TECHNOLOGIES LLC,1211 31ST ST. NW,WASHINGTON,DC,20007,UNITED STATES OF AMERICA,2022953000,Y,"BURKE, KAREN",...,Year,Y,N,Y,N,WASHINGTON,NaN,DC,20007,0
3,20,INFO SERVICES LLC,17177 NORTH LAUREL PARK DR,LIVONIA,MI,48152,UNITED STATES OF AMERICA,7343776007,N,",",...,Year,Y,N,Y,N,JERSEY CITY,HUDSON,NJ,07302,0
4,18,BB&T CORPORATION,223 WEST NASH STREET,WILSON,NC,27893,UNITED STATES OF AMERICA,2522462031,Y,"SCOFIELD, EILEEN",...,Year,N,N,N,Y,NEW YORK,NEW YORK,NY,10036,0


## Remove other correlated columns

In [15]:
reduced_data.columns

Index(['EMPLOYMENT_END_DATE', 'EMPLOYER_NAME', 'EMPLOYER_ADDRESS',
       'EMPLOYER_CITY', 'EMPLOYER_STATE', 'EMPLOYER_POSTAL_CODE',
       'EMPLOYER_COUNTRY', 'EMPLOYER_PHONE', 'AGENT_REPRESENTING_EMPLOYER',
       'AGENT_ATTORNEY_NAME', 'JOB_TITLE', 'SOC_CODE', 'SOC_NAME',
       'NAICS_CODE', 'TOTAL_WORKERS', 'NEW_EMPLOYMENT', 'CONTINUED_EMPLOYMENT',
       'CHANGE_PREVIOUS_EMPLOYMENT', 'NEW_CONCURRENT_EMPLOYMENT',
       'CHANGE_EMPLOYER', 'AMENDED_PETITION', 'FULL_TIME_POSITION',
       'PREVAILING_WAGE', 'PW_UNIT_OF_PAY', 'PW_WAGE_LEVEL', 'PW_SOURCE',
       'PW_SOURCE_YEAR', 'PW_SOURCE_OTHER', 'WAGE_RATE_OF_PAY_TO',
       'WAGE_UNIT_OF_PAY', 'H1B_DEPENDENT', 'WILLFUL_VIOLATOR', 'SUPPORT_H1B',
       'LABOR_CON_AGREE', 'WORKSITE_CITY', 'WORKSITE_COUNTY', 'WORKSITE_STATE',
       'WORKSITE_POSTAL_CODE', 'label'],
      dtype='object')

In [16]:
employer_related = ['EMPLOYER_ADDRESS', 'EMPLOYER_CITY', 'EMPLOYER_STATE',
                   'EMPLOYER_POSTAL_CODE', 'EMPLOYER_COUNTRY', 'EMPLOYER_PHONE',
                   'AGENT_REPRESENTING_EMPLOYER', 'AGENT_ATTORNEY_NAME']
job_related = ['JOB_TITLE', 'SOC_CODE', 'NAICS_CODE']
wordsite_related = ['WORKSITE_CITY', 'WORKSITE_COUNTY', 'WORKSITE_STATE',
                   'WORKSITE_POSTAL_CODE']

In [17]:
for col in employer_related:
    reduced_data = reduced_data.drop(col, axis=1)

for col in job_related:
    reduced_data = reduced_data.drop(col, axis=1)
    
for col in wordsite_related:
    reduced_data = reduced_data.drop(col, axis=1)

reduced_data.columns

Index(['EMPLOYMENT_END_DATE', 'EMPLOYER_NAME', 'SOC_NAME', 'TOTAL_WORKERS',
       'NEW_EMPLOYMENT', 'CONTINUED_EMPLOYMENT', 'CHANGE_PREVIOUS_EMPLOYMENT',
       'NEW_CONCURRENT_EMPLOYMENT', 'CHANGE_EMPLOYER', 'AMENDED_PETITION',
       'FULL_TIME_POSITION', 'PREVAILING_WAGE', 'PW_UNIT_OF_PAY',
       'PW_WAGE_LEVEL', 'PW_SOURCE', 'PW_SOURCE_YEAR', 'PW_SOURCE_OTHER',
       'WAGE_RATE_OF_PAY_TO', 'WAGE_UNIT_OF_PAY', 'H1B_DEPENDENT',
       'WILLFUL_VIOLATOR', 'SUPPORT_H1B', 'LABOR_CON_AGREE', 'label'],
      dtype='object')

In [18]:
reduced_data.shape

(1249823, 24)

In [19]:
reduced_data.to_csv('reduced_data.csv', index=False)

# Feature Engineering 2.0 Generate Categorized Data

In [6]:
data = pd.read_csv('reduced_data.csv')
data = data.drop('PW_SOURCE_OTHER', axis=1)
print(data.shape)
print(data.columns)
print(data.dtypes)
data.head()

(1249823, 23)
Index(['EMPLOYMENT_END_DATE', 'EMPLOYER_NAME', 'SOC_NAME', 'TOTAL_WORKERS',
       'NEW_EMPLOYMENT', 'CONTINUED_EMPLOYMENT', 'CHANGE_PREVIOUS_EMPLOYMENT',
       'NEW_CONCURRENT_EMPLOYMENT', 'CHANGE_EMPLOYER', 'AMENDED_PETITION',
       'FULL_TIME_POSITION', 'PREVAILING_WAGE', 'PW_UNIT_OF_PAY',
       'PW_WAGE_LEVEL', 'PW_SOURCE', 'PW_SOURCE_YEAR', 'WAGE_RATE_OF_PAY_TO',
       'WAGE_UNIT_OF_PAY', 'H1B_DEPENDENT', 'WILLFUL_VIOLATOR', 'SUPPORT_H1B',
       'LABOR_CON_AGREE', 'label'],
      dtype='object')
EMPLOYMENT_END_DATE             int64
EMPLOYER_NAME                  object
SOC_NAME                       object
TOTAL_WORKERS                   int64
NEW_EMPLOYMENT                  int64
CONTINUED_EMPLOYMENT            int64
CHANGE_PREVIOUS_EMPLOYMENT      int64
NEW_CONCURRENT_EMPLOYMENT       int64
CHANGE_EMPLOYER                 int64
AMENDED_PETITION                int64
FULL_TIME_POSITION             object
PREVAILING_WAGE               float64
PW_UNIT_OF_PAY     

,EMPLOYMENT_END_DATE,EMPLOYER_NAME,SOC_NAME,TOTAL_WORKERS,NEW_EMPLOYMENT,CONTINUED_EMPLOYMENT,CHANGE_PREVIOUS_EMPLOYMENT,NEW_CONCURRENT_EMPLOYMENT,CHANGE_EMPLOYER,AMENDED_PETITION,...,PW_WAGE_LEVEL,PW_SOURCE,PW_SOURCE_YEAR,WAGE_RATE_OF_PAY_TO,WAGE_UNIT_OF_PAY,H1B_DEPENDENT,WILLFUL_VIOLATOR,SUPPORT_H1B,LABOR_CON_AGREE,label
0,19,DISCOVER PRODUCTS INC.,COMPUTER SYSTEMS ANALYSTS,1,1,0,0,0,0,0,...,Level I,OES,2015.0,67320.0,Year,N,N,N,Y,0
1,19,DFS SERVICES LLC,OPERATIONS RESEARCH ANALYSTS,1,1,0,0,0,0,0,...,NaN,Other,2015.0,57200.0,Year,N,N,N,Y,0
2,19,EASTBANC TECHNOLOGIES LLC,COMPUTER PROGRAMMERS,2,2,0,0,0,0,0,...,Level II,OES,2015.0,0.0,Year,Y,N,Y,N,0
3,20,INFO SERVICES LLC,"COMPUTER OCCUPATIONS, ALL OTHER",1,1,0,0,0,0,0,...,Level III,OES,2016.0,0.0,Year,Y,N,Y,N,0
4,18,BB&T CORPORATION,CREDIT ANALYSTS,1,0,0,0,0,1,0,...,Level III,OES,2015.0,0.0,Year,N,N,N,Y,0


## Convert Yes/No columns to 1/0

In [8]:
YN_Column = ['FULL_TIME_POSITION', 'H1B_DEPENDENT', 'SUPPORT_H1B',
            'WILLFUL_VIOLATOR', 'SUPPORT_H1B', 'LABOR_CON_AGREE']

In [9]:
def convert_YN(df, col_name):
    result = []
    col = df[col_name]
    for c in col:
        if type(c) == str and c == 'Y':
            result.append(1)
        else:
            result.append(0)
    return result

In [10]:
for yn in YN_Column:
    data[yn] = convert_YN(data, yn)
data.head()

,EMPLOYMENT_END_DATE,EMPLOYER_NAME,SOC_NAME,TOTAL_WORKERS,NEW_EMPLOYMENT,CONTINUED_EMPLOYMENT,CHANGE_PREVIOUS_EMPLOYMENT,NEW_CONCURRENT_EMPLOYMENT,CHANGE_EMPLOYER,AMENDED_PETITION,...,PW_WAGE_LEVEL,PW_SOURCE,PW_SOURCE_YEAR,WAGE_RATE_OF_PAY_TO,WAGE_UNIT_OF_PAY,H1B_DEPENDENT,WILLFUL_VIOLATOR,SUPPORT_H1B,LABOR_CON_AGREE,label
0,19,DISCOVER PRODUCTS INC.,COMPUTER SYSTEMS ANALYSTS,1,1,0,0,0,0,0,...,Level I,OES,2015.0,67320.0,Year,0,0,0,1,0
1,19,DFS SERVICES LLC,OPERATIONS RESEARCH ANALYSTS,1,1,0,0,0,0,0,...,NaN,Other,2015.0,57200.0,Year,0,0,0,1,0
2,19,EASTBANC TECHNOLOGIES LLC,COMPUTER PROGRAMMERS,2,2,0,0,0,0,0,...,Level II,OES,2015.0,0.0,Year,1,0,0,0,0
3,20,INFO SERVICES LLC,"COMPUTER OCCUPATIONS, ALL OTHER",1,1,0,0,0,0,0,...,Level III,OES,2016.0,0.0,Year,1,0,0,0,0
4,18,BB&T CORPORATION,CREDIT ANALYSTS,1,0,0,0,0,1,0,...,Level III,OES,2015.0,0.0,Year,0,0,0,1,0


In [12]:
data.dtypes

EMPLOYMENT_END_DATE             int64
EMPLOYER_NAME                  object
SOC_NAME                       object
TOTAL_WORKERS                   int64
NEW_EMPLOYMENT                  int64
CONTINUED_EMPLOYMENT            int64
CHANGE_PREVIOUS_EMPLOYMENT      int64
NEW_CONCURRENT_EMPLOYMENT       int64
CHANGE_EMPLOYER                 int64
AMENDED_PETITION                int64
FULL_TIME_POSITION              int64
PREVAILING_WAGE               float64
PW_UNIT_OF_PAY                 object
PW_WAGE_LEVEL                  object
PW_SOURCE                      object
PW_SOURCE_YEAR                float64
WAGE_RATE_OF_PAY_TO           float64
WAGE_UNIT_OF_PAY               object
H1B_DEPENDENT                   int64
WILLFUL_VIOLATOR                int64
SUPPORT_H1B                     int64
LABOR_CON_AGREE                 int64
label                           int64
dtype: object

## Convert wages to same unit

In [15]:
def convert_wage(df, wages, units):
    wage = df[wages]
    unit = df[units]
    result = []
    for i in range(0, len(wage)):
        w = wage[i]
        u = unit[i]
        if u == 'Bi-Weekly':
            result.append(w * 26)
        elif u == 'Hour':
            result.append(w * 40 * 52)
        elif u == 'Month':
            result.append(w * 12)
        elif u == 'Week':
            result.append(w * 52)
        else:
            result.append(w)
    return result
    

In [16]:
data['PREVAILING_WAGE'] = convert_wage(data, 'PREVAILING_WAGE', 'PW_UNIT_OF_PAY')
data['WAGE_RATE_OF_PAY_TO'] = convert_wage(data, 'WAGE_RATE_OF_PAY_TO', 'WAGE_UNIT_OF_PAY')
data.head()

,EMPLOYMENT_END_DATE,EMPLOYER_NAME,SOC_NAME,TOTAL_WORKERS,NEW_EMPLOYMENT,CONTINUED_EMPLOYMENT,CHANGE_PREVIOUS_EMPLOYMENT,NEW_CONCURRENT_EMPLOYMENT,CHANGE_EMPLOYER,AMENDED_PETITION,...,PW_WAGE_LEVEL,PW_SOURCE,PW_SOURCE_YEAR,WAGE_RATE_OF_PAY_TO,WAGE_UNIT_OF_PAY,H1B_DEPENDENT,WILLFUL_VIOLATOR,SUPPORT_H1B,LABOR_CON_AGREE,label
0,19,DISCOVER PRODUCTS INC.,COMPUTER SYSTEMS ANALYSTS,1,1,0,0,0,0,0,...,Level I,OES,2015.0,67320.0,Year,0,0,0,1,0
1,19,DFS SERVICES LLC,OPERATIONS RESEARCH ANALYSTS,1,1,0,0,0,0,0,...,NaN,Other,2015.0,57200.0,Year,0,0,0,1,0
2,19,EASTBANC TECHNOLOGIES LLC,COMPUTER PROGRAMMERS,2,2,0,0,0,0,0,...,Level II,OES,2015.0,0.0,Year,1,0,0,0,0
3,20,INFO SERVICES LLC,"COMPUTER OCCUPATIONS, ALL OTHER",1,1,0,0,0,0,0,...,Level III,OES,2016.0,0.0,Year,1,0,0,0,0
4,18,BB&T CORPORATION,CREDIT ANALYSTS,1,0,0,0,0,1,0,...,Level III,OES,2015.0,0.0,Year,0,0,0,1,0


In [20]:
data = data.drop('PW_UNIT_OF_PAY', axis=1)
data = data.drop('WAGE_UNIT_OF_PAY', axis=1)
data.columns

Index(['EMPLOYMENT_END_DATE', 'EMPLOYER_NAME', 'SOC_NAME', 'TOTAL_WORKERS',
       'NEW_EMPLOYMENT', 'CONTINUED_EMPLOYMENT', 'CHANGE_PREVIOUS_EMPLOYMENT',
       'NEW_CONCURRENT_EMPLOYMENT', 'CHANGE_EMPLOYER', 'AMENDED_PETITION',
       'FULL_TIME_POSITION', 'PREVAILING_WAGE', 'PW_WAGE_LEVEL', 'PW_SOURCE',
       'PW_SOURCE_YEAR', 'WAGE_RATE_OF_PAY_TO', 'H1B_DEPENDENT',
       'WILLFUL_VIOLATOR', 'SUPPORT_H1B', 'LABOR_CON_AGREE', 'label'],
      dtype='object')

## Dealing with Flags

In [24]:
flags = ['NEW_EMPLOYMENT', 'CONTINUED_EMPLOYMENT', 'CHANGE_PREVIOUS_EMPLOYMENT',
       'NEW_CONCURRENT_EMPLOYMENT', 'CHANGE_EMPLOYER', 'AMENDED_PETITION']

In [22]:
def gen_flag(df, col_name):
    result = []
    for c in df[col_name]:
        if c == 0:
            result.append(0)
        else:
            result.append(1)
    return result

In [25]:
for f in flags:
    data[f] = gen_flag(data, f)
data.head()

,EMPLOYMENT_END_DATE,EMPLOYER_NAME,SOC_NAME,TOTAL_WORKERS,NEW_EMPLOYMENT,CONTINUED_EMPLOYMENT,CHANGE_PREVIOUS_EMPLOYMENT,NEW_CONCURRENT_EMPLOYMENT,CHANGE_EMPLOYER,AMENDED_PETITION,...,PREVAILING_WAGE,PW_WAGE_LEVEL,PW_SOURCE,PW_SOURCE_YEAR,WAGE_RATE_OF_PAY_TO,H1B_DEPENDENT,WILLFUL_VIOLATOR,SUPPORT_H1B,LABOR_CON_AGREE,label
0,19,DISCOVER PRODUCTS INC.,COMPUTER SYSTEMS ANALYSTS,1,1,0,0,0,0,0,...,59197.0,Level I,OES,2015.0,67320.0,0,0,0,1,0
1,19,DFS SERVICES LLC,OPERATIONS RESEARCH ANALYSTS,1,1,0,0,0,0,0,...,49800.0,NaN,Other,2015.0,57200.0,0,0,0,1,0
2,19,EASTBANC TECHNOLOGIES LLC,COMPUTER PROGRAMMERS,2,1,0,0,0,0,0,...,76502.0,Level II,OES,2015.0,0.0,1,0,0,0,0
3,20,INFO SERVICES LLC,"COMPUTER OCCUPATIONS, ALL OTHER",1,1,0,0,0,0,0,...,90376.0,Level III,OES,2016.0,0.0,1,0,0,0,0
4,18,BB&T CORPORATION,CREDIT ANALYSTS,1,0,0,0,0,1,0,...,116605.0,Level III,OES,2015.0,0.0,0,0,0,1,0


### store categorized data

In [29]:
data = data.dropna()
data.shape

(1093097, 21)

In [31]:
data.to_csv('categorized.csv', index=False)

# Feature Engineering 3.0 Generate Numerical Data

In [20]:
data = pd.read_csv('categorized.csv')
data.head()

,EMPLOYMENT_END_DATE,EMPLOYER_NAME,SOC_NAME,TOTAL_WORKERS,NEW_EMPLOYMENT,CONTINUED_EMPLOYMENT,CHANGE_PREVIOUS_EMPLOYMENT,NEW_CONCURRENT_EMPLOYMENT,CHANGE_EMPLOYER,AMENDED_PETITION,...,PREVAILING_WAGE,PW_WAGE_LEVEL,PW_SOURCE,PW_SOURCE_YEAR,WAGE_RATE_OF_PAY_TO,H1B_DEPENDENT,WILLFUL_VIOLATOR,SUPPORT_H1B,LABOR_CON_AGREE,label
0,19,DISCOVER PRODUCTS INC.,COMPUTER SYSTEMS ANALYSTS,1,1,0,0,0,0,0,...,59197.0,Level I,OES,2015.0,67320.0,0,0,0,1,0
1,19,EASTBANC TECHNOLOGIES LLC,COMPUTER PROGRAMMERS,2,1,0,0,0,0,0,...,76502.0,Level II,OES,2015.0,0.0,1,0,0,0,0
2,20,INFO SERVICES LLC,"COMPUTER OCCUPATIONS, ALL OTHER",1,1,0,0,0,0,0,...,90376.0,Level III,OES,2016.0,0.0,1,0,0,0,0
3,18,BB&T CORPORATION,CREDIT ANALYSTS,1,0,0,0,0,1,0,...,116605.0,Level III,OES,2015.0,0.0,0,0,0,1,0
4,18,"SUNTRUST BANKS, INC.","FINANCIAL SPECIALISTS, ALL OTHER",1,1,0,0,0,0,0,...,59405.0,Level III,OES,2015.0,0.0,0,0,0,1,0


In [21]:
print(data.shape)
data.dtypes

(1093097, 21)


EMPLOYMENT_END_DATE             int64
EMPLOYER_NAME                  object
SOC_NAME                       object
TOTAL_WORKERS                   int64
NEW_EMPLOYMENT                  int64
CONTINUED_EMPLOYMENT            int64
CHANGE_PREVIOUS_EMPLOYMENT      int64
NEW_CONCURRENT_EMPLOYMENT       int64
CHANGE_EMPLOYER                 int64
AMENDED_PETITION                int64
FULL_TIME_POSITION              int64
PREVAILING_WAGE               float64
PW_WAGE_LEVEL                  object
PW_SOURCE                      object
PW_SOURCE_YEAR                float64
WAGE_RATE_OF_PAY_TO           float64
H1B_DEPENDENT                   int64
WILLFUL_VIOLATOR                int64
SUPPORT_H1B                     int64
LABOR_CON_AGREE                 int64
label                           int64
dtype: object

### drop objects that can't be normalize

In [22]:
data = data.drop('EMPLOYER_NAME', axis=1).drop('SOC_NAME', axis=1)
data.dtypes

EMPLOYMENT_END_DATE             int64
TOTAL_WORKERS                   int64
NEW_EMPLOYMENT                  int64
CONTINUED_EMPLOYMENT            int64
CHANGE_PREVIOUS_EMPLOYMENT      int64
NEW_CONCURRENT_EMPLOYMENT       int64
CHANGE_EMPLOYER                 int64
AMENDED_PETITION                int64
FULL_TIME_POSITION              int64
PREVAILING_WAGE               float64
PW_WAGE_LEVEL                  object
PW_SOURCE                      object
PW_SOURCE_YEAR                float64
WAGE_RATE_OF_PAY_TO           float64
H1B_DEPENDENT                   int64
WILLFUL_VIOLATOR                int64
SUPPORT_H1B                     int64
LABOR_CON_AGREE                 int64
label                           int64
dtype: object

## One Hot Encoding on PW_WAGE_LEVEL and PW_SOURCE

In [17]:
wl = data.groupby('PW_WAGE_LEVEL')
for n, d in wl:
    print(n)

Level I
Level II
Level III
Level IV


In [18]:
source = data.groupby('PW_SOURCE')
for n, d in source:
    print(n)

CBA
DBA
OES
Other
SCA


In [23]:
wage_level_1 = []
wage_level_2 = []
wage_level_3 = []
wage_level_4 = []
for i in data['PW_WAGE_LEVEL']:
    if i == 'Level I':
        wage_level_1.append(1)
        wage_level_2.append(0)
        wage_level_3.append(0)
        wage_level_4.append(0)
    elif i == 'Level II':
        wage_level_1.append(0)
        wage_level_2.append(1)
        wage_level_3.append(0)
        wage_level_4.append(0)
    elif i == 'Level III':
        wage_level_1.append(0)
        wage_level_2.append(0)
        wage_level_3.append(1)
        wage_level_4.append(0)
    else:
        wage_level_1.append(0)
        wage_level_2.append(0)
        wage_level_3.append(0)
        wage_level_4.append(1)

In [24]:
data['WAGE_LEVEL_1'] = wage_level_1
data['WAGE_LEVEL_2'] = wage_level_2
data['WAGE_LEVEL_3'] = wage_level_3
data['WAGE_LEVEL_4'] = wage_level_4
data = data.drop('PW_WAGE_LEVEL', axis=1)
data.head()

,EMPLOYMENT_END_DATE,TOTAL_WORKERS,NEW_EMPLOYMENT,CONTINUED_EMPLOYMENT,CHANGE_PREVIOUS_EMPLOYMENT,NEW_CONCURRENT_EMPLOYMENT,CHANGE_EMPLOYER,AMENDED_PETITION,FULL_TIME_POSITION,PREVAILING_WAGE,...,WAGE_RATE_OF_PAY_TO,H1B_DEPENDENT,WILLFUL_VIOLATOR,SUPPORT_H1B,LABOR_CON_AGREE,label,WAGE_LEVEL_1,WAGE_LEVEL_2,WAGE_LEVEL_3,WAGE_LEVEL_4
0,19,1,1,0,0,0,0,0,1,59197.0,...,67320.0,0,0,0,1,0,1,0,0,0
1,19,2,1,0,0,0,0,0,1,76502.0,...,0.0,1,0,0,0,0,0,1,0,0
2,20,1,1,0,0,0,0,0,1,90376.0,...,0.0,1,0,0,0,0,0,0,1,0
3,18,1,0,0,0,0,1,0,1,116605.0,...,0.0,0,0,0,1,0,0,0,1,0
4,18,1,1,0,0,0,0,0,1,59405.0,...,0.0,0,0,0,1,0,0,0,1,0


In [26]:

CBA = []
DBA = []
OES = []
Other_pw_source = []
SCA = []
for i in data['PW_SOURCE']:
    if i == 'CBA':
        CBA.append(1)
        DBA.append(0)
        OES.append(0)
        Other_pw_source.append(0)
        SCA.append(0)
    elif i == 'DBA':
        CBA.append(0)
        DBA.append(1)
        OES.append(0)
        Other_pw_source.append(0)
        SCA.append(0)
    elif i == 'OES':
        CBA.append(0)
        DBA.append(0)
        OES.append(1)
        Other_pw_source.append(0)
        SCA.append(0)
    elif i == 'SCA':
        CBA.append(0)
        DBA.append(0)
        OES.append(0)
        Other_pw_source.append(0)
        SCA.append(1)
    else:
        CBA.append(0)
        DBA.append(0)
        OES.append(0)
        Other_pw_source.append(1)
        SCA.append(0)
    

In [27]:
data['CBA'] = CBA
data['DBA'] = DBA
data['OES'] = OES
data['SCA'] = SCA
data['OTHER_PW_SOURCE'] = Other_pw_source
data = data.drop('PW_SOURCE', axis=1)
data.head()

,EMPLOYMENT_END_DATE,TOTAL_WORKERS,NEW_EMPLOYMENT,CONTINUED_EMPLOYMENT,CHANGE_PREVIOUS_EMPLOYMENT,NEW_CONCURRENT_EMPLOYMENT,CHANGE_EMPLOYER,AMENDED_PETITION,FULL_TIME_POSITION,PREVAILING_WAGE,...,label,WAGE_LEVEL_1,WAGE_LEVEL_2,WAGE_LEVEL_3,WAGE_LEVEL_4,CBA,DBA,OES,SCA,OTHER_PW_SOURCE
0,19,1,1,0,0,0,0,0,1,59197.0,...,0,1,0,0,0,0,0,1,0,0
1,19,2,1,0,0,0,0,0,1,76502.0,...,0,0,1,0,0,0,0,1,0,0
2,20,1,1,0,0,0,0,0,1,90376.0,...,0,0,0,1,0,0,0,1,0,0
3,18,1,0,0,0,0,1,0,1,116605.0,...,0,0,0,1,0,0,0,1,0,0
4,18,1,1,0,0,0,0,0,1,59405.0,...,0,0,0,1,0,0,0,1,0,0


In [28]:
data.dtypes

EMPLOYMENT_END_DATE             int64
TOTAL_WORKERS                   int64
NEW_EMPLOYMENT                  int64
CONTINUED_EMPLOYMENT            int64
CHANGE_PREVIOUS_EMPLOYMENT      int64
NEW_CONCURRENT_EMPLOYMENT       int64
CHANGE_EMPLOYER                 int64
AMENDED_PETITION                int64
FULL_TIME_POSITION              int64
PREVAILING_WAGE               float64
PW_SOURCE_YEAR                float64
WAGE_RATE_OF_PAY_TO           float64
H1B_DEPENDENT                   int64
WILLFUL_VIOLATOR                int64
SUPPORT_H1B                     int64
LABOR_CON_AGREE                 int64
label                           int64
WAGE_LEVEL_1                    int64
WAGE_LEVEL_2                    int64
WAGE_LEVEL_3                    int64
WAGE_LEVEL_4                    int64
CBA                             int64
DBA                             int64
OES                             int64
SCA                             int64
OTHER_PW_SOURCE                 int64
dtype: objec

## Remove records that are out of 3 standard deviation

In [29]:
data.shape

(1093097, 26)

In [30]:
cols = ['PREVAILING_WAGE', 'PW_SOURCE_YEAR', 'WAGE_RATE_OF_PAY_TO']
for c in cols:
    mean = data[c].mean()
    std = data[c].std()
    data = data[data[c] < mean + 3 * std]
    data = data[data[c] > mean - 3 * std]
print(data.shape)
data.head()

(1092830, 26)


,EMPLOYMENT_END_DATE,TOTAL_WORKERS,NEW_EMPLOYMENT,CONTINUED_EMPLOYMENT,CHANGE_PREVIOUS_EMPLOYMENT,NEW_CONCURRENT_EMPLOYMENT,CHANGE_EMPLOYER,AMENDED_PETITION,FULL_TIME_POSITION,PREVAILING_WAGE,...,label,WAGE_LEVEL_1,WAGE_LEVEL_2,WAGE_LEVEL_3,WAGE_LEVEL_4,CBA,DBA,OES,SCA,OTHER_PW_SOURCE
0,19,1,1,0,0,0,0,0,1,59197.0,...,0,1,0,0,0,0,0,1,0,0
1,19,2,1,0,0,0,0,0,1,76502.0,...,0,0,1,0,0,0,0,1,0,0
2,20,1,1,0,0,0,0,0,1,90376.0,...,0,0,0,1,0,0,0,1,0,0
3,18,1,0,0,0,0,1,0,1,116605.0,...,0,0,0,1,0,0,0,1,0,0
4,18,1,1,0,0,0,0,0,1,59405.0,...,0,0,0,1,0,0,0,1,0,0


### store numerical data

In [31]:
data.to_csv('numerical.csv', index=False)